<h1> Biol 696 Final Project: Machine Learning Implementation of Retention Time Data to Assist in Metabolomic Data Processing </h

Goal of Project: Take data from a publically available database(PredRet), preprocess data to create a data set that a nueral network can use, and tune parameters to try to find the most efficient model

<h2> Part 1: Taking rudimentary data, process using open source chemoinformatics packages, pubchempy and rdkit, and create a machine learning dataset</h2>

NOTE: In an attempt to make this notebook readable, I pickled the dataframes so you can see what I did without actually having to run the code. I will include the not outputted code in the notebook, and if you want to see it in action, will happily stop by and show you in real time!

<h3> Intital Dataset </h3>

In [7]:
import pandas as pd

initialdata = pd.read_csv('retention_time_testdata.csv')
print(initialdata.head())

   Unnamed: 0                       Name    System Username       RT  Pubchem  \
0           1                epicatechin  FEM_long      jan  20.4650      NaN   
1           2                    chrysin  FEM_long      jan  40.8800      NaN   
2           3        epicatechin gallate  FEM_long      jan  24.7200      NaN   
3           4           epigallocatechin  FEM_long      jan  16.5050      NaN   
4           5  epigallo-catechin gallate  FEM_long      jan  22.2925      NaN   

                                               InChI            Time  
0  InChI=1S/C15H14O6/c16-8-4-11(18)9-6-13(20)15(2...  5/12/2015 4:32  
1  InChI=1S/C15H10O4/c16-10-6-11(17)15-12(18)8-13...   9/2/2014 5:20  
2  InChI=1S/C22H18O10/c23-11-6-14(25)12-8-19(32-2...   9/2/2014 5:20  
3  InChI=1S/C15H14O7/c16-7-3-9(17)8-5-12(20)15(22...   9/2/2014 5:20  
4  InChI=1S/C22H18O11/c23-10-5-12(24)11-7-18(33-2...   6/8/2015 7:19  


As shown, the data is not even close to being ready for implementation. After EXTENSIVE research, I found the chemoinformatic packages mentioned above. Without going too in depth, I essentially parsed out the InChI column(a compound identifier), put it into pubchempy to get some chemical properties(SMILES key being one), then I put the Smiles key into RDkit to pull an extensive numerical list of chemical properties. The code is as follows, but it will most likely error/not run correctly on your system. It's a definition, so keeping the last line uncommented is recommended 

In [8]:
from rdkit import Chem, DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import MACCSkeys
import pubchempy as pcp
import pandas as pd
from rdkit.Chem.EState import Fingerprinter
import numpy as np
from rdkit.Chem import Descriptors
import pickle
#example testing: coing from inchi to testable features

def fps_plus_mw(mol):
    return np.append(Fingerprinter.FingerprintMol(mol),Descriptors.MolWt(mol))

def mol_prop_gen(filename,outputpickle):
#loading initial data
    dataframe = pd.read_csv(filename)
    newdf = pd.DataFrame()
    for index, row in dataframe.iterrows():
        if index == 0:
            inchi = row['InChI']
            cmpd = pcp.get_compounds(inchi,'inchi')
            props = cmpd[0].to_dict(properties=['cactvs_fingerprint','isomeric_smiles', 'xlogp', 'rotatable_bond_count','charge','complexity','exact_mass','fingerprint'])
            smiles=props['isomeric_smiles']
            props['mol']=Chem.MolFromSmiles(smiles)
            props['RT'] = row['RT']
            desc = np.array(fps_plus_mw(props['mol']))
            descdf = pd.DataFrame(desc)
            descdf = descdf.T
            descdf.reindex([index])
            newdf=pd.DataFrame(props,index=[index])
            finaldf=pd.concat([descdf,newdf],axis=1)
        else:
            inchi = row['InChI']
        try:
            cmpd = pcp.get_compounds(inchi,'inchi')
        except:
            print('line bypassed')
            pass
        try:
            props = cmpd[0].to_dict(properties=['cactvs_fingerprint','isomeric_smiles', 'xlogp', 'rotatable_bond_count','charge','complexity','exact_mass','fingerprint'])
        except:
            print('line bypassed')
            pass
        smiles=props['isomeric_smiles']
        props['mol']=Chem.MolFromSmiles(smiles)
        props['RT'] = row['RT']
        newdf=pd.DataFrame(props,index=[index])
        desc = np.array(fps_plus_mw(props['mol']))
        cols=range(len(desc))
        descdf=pd.DataFrame(desc)
        descdf = descdf.T
        descdf.index = [index]
#        descdf = descdf.T
#        descdf = pd.DataFrame(descdf, index=[index])
        interdf = pd.concat([descdf,newdf],axis=1)
        finaldf = finaldf.append(interdf)
        print('on index ' + str(index+1) + ' of ' + str(len(dataframe)))
        finaldf.to_pickle(outputpickler)


#mol_prop_gen('retention_time_testdatashort.csv','test.pickle')

In [9]:
#RUN THIS CELL Dr. Kelley to see data
import pickle

testdata = pd.read_pickle('my_df.pickle')
print(testdata.head())

     0    1    2    3    4    5    6    7    8    9   ...     \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   ...      
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   ...      
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   ...      
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   ...      
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   ...      

                                  cactvs_fingerprint  \
0  1100000001110000001110000000000000000000000000...   
1  1100000001110000001110000000000000000000000000...   
2  1110000001111000001111000000000000000000000000...   
3  1100000001110000001110000000000000000000000000...   
4  1110000001111000001111000000000000000000000000...   

                                     isomeric_smiles  xlogp  \
0      C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C=C3)O)O)O    0.4   
1          C1=CC=C(C=C1)C2=CC(=O)C3=C(C=C(C=C3O2)O)O    2.1   
2  C1C(C(OC2=CC(=CC(=C21)O)O)C3=CC(=C(C=C3)O)O)OC...    1.5   
3   C1C(C(OC2=CC(=CC(=C21)

As you can see, this is a lot more computer readable and more suited for a nueral network. Strings and objects, such as column isomeric_smiles and mol are dropped later on, but are nice identifiers for long term use of this data frame. Column RT is the Retention Time, which we will be training on and trying to predict.

<h2>PART 2: Running Nueral Networks</h2>
Not entirely sure how this will run on your system Dr. Kelley, and it will take a long time to run. Will include the code if you really want to run it, but I will discuss the results

<h3> Here's the final code I used to test the best optimizing function for the NN. I suggest not running this in jupyter, but will attach the python file with the submission. Once again, will be happy to do a tutorial in real time. Note How I picked out the results dataframe, which we will look over </h3>

In [ ]:
#models in a loop
#sgd = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
#rms = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
#ada = keras.optimizers.Adagrad(lr=0.001, epsilon=None, decay=0.0)
#adad = keras.optimizers.Adadelta(lr=0.001, rho=0.95, epsilon=None, decay=0.0)
#adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#adamax = keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
#nadam = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
#epochs = 10000
#finalresults = pd.DataFrame()
#optimizers = [ada,adad,adam,adamax,nadam,rms,sgd]
#accuracy = []
#for count,o in enumerate(optimizers,0):
#    model = Sequential()
#    model.add(Dense(output_dim=5, input_dim=xvalues.shape[1]))
#    model.add(Activation("sigmoid"))
#    model.add(Dense(output_dim=1))
#    model.add(Activation("linear"))
#    model.compile(loss='mean_squared_error', optimizer=o)
#    history = model.fit(xtrain, ytrain, nb_epoch=epochs, batch_size=32)
#    y_pred = model.predict(xtest)
#    y_preddf = pd.DataFrame(y_pred)
#    ytest=ytest.reset_index()
#    ytest = ytest.drop(['index'],axis=1)
#    interdf = pd.concat([ytest,y_preddf],axis=1)
#    correct = 0 
#    incorrect = 0
#    listofper = []
#    listofpre = []
#    for index,row in interdf.iterrows():
#        pred = row[0]
#        listofpre.append(pred)
#        actu = row['RT']
#        percent = ((pred-actu)/actu) * 100
#        diff = pred - actu
#        listofper.append(percent)
#        if diff < 5.00 and diff > -5.00:
#            correct += 1
#        else:
#            incorrect += 1
#        samples = len(ytest)
#        pos = (correct/samples) *100
#    accuracy.append([count,pos])
#    resultsdf['cv ' + str(count)] = listofper
#    resultsdf['predictions ' + str(count)] = listofpre
#print(accuracy)   
#resultsdf.to_pickle('results_df.pickle')   

<h2> Part 3: Results </h2>

In [5]:
import pickle
with open('accuracy.pkl', 'rb') as f:
   acc = pickle.load(f)
print(acc)

([0, 31.451612903225808], [1, 30.64516129032258], [2, 71.7741935483871], [3, 65.32258064516128], [4, 64.51612903225806], [5, 66.93548387096774], [6, 75.80645161290323])


As shown, index 6 is our most "accurate" optimizing funcion. these indexes correspond to the index in optimizers = [ada,adad,adam,adamax,nadam,rms,sgd]. so our SGD optimizer was out best function, and overall my model performed a lot better than I expected! 
Some notes on what I called a "success". I initally tried to use percents. a guess has to be within 5%. That was very troublesome for all the compounds with low retention times(if RT is 2, and the guess is 2.8, it's pretty close but it's 30% off)!
So a "correct" guess is something that is within 5 mins of the actual value. Although this is really high of a range, this was more of a proof of concept study and it shows the network can be remotely close to the actual value. 

In [6]:
import pickle

testdata = pd.read_pickle('results_df.pickle')
print(testdata)

          RT          0  CVsimplesNN       cv 0  predictions 0       cv 1  \
0     2.3850   1.162806   -51.245035 -13.166974       2.070968 -29.540372   
1    34.7800  34.321083    -1.319485 -89.978984       3.485309 -94.415099   
2    27.8200  26.250219    -5.642634 -88.353149       3.240154 -93.308797   
3    32.6475  33.724678     3.299420 -88.671725       3.698399 -93.598508   
4    16.9000  22.545456    33.405065 -82.912919       2.887717 -89.493549   
5    59.5400  57.148415    -4.016771 -93.847573       3.663155 -96.526030   
6     2.1000   3.821881    81.994336 -37.535610       1.311752 -15.583270   
7    52.8000  55.108494     4.372147 -93.269020       3.553958 -96.395347   
8     1.4000   0.986148   -29.560840 -48.183456       0.725432 -21.200897   
9    42.2000  39.286201    -6.904736 -91.297339       3.672523 -95.498609   
10    2.2000  12.565017   471.137125  -9.944166       1.981228 -27.093950   
11   21.5950  19.656458    -8.976810 -84.039863       3.446592 -91.890412   

Above, you can see my results data frame. It uses the same indexes as accuracy, and inclused the actual values, the predictions, and the cv. 

<h2> Conclusion </h2>
Overall, the time put into this project was worth it. I think this will prove as a straight forward proof of concept for my PI(Dr. Forsberg) that we should try to implement a similar sort of algorithm in out lab. The eventual goal will be to have our model help identify whether or not our guess of what a compound is and where it elutes matches up reasonablly well with our model. 

<h3> Future Works </h3>
Implementing into metabolomics workflow, creating a model based on compunds we have actually run on our system!
Thanks For a great semester Scott Kelley! Hope to see you around!